In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import dataset
from bs4 import BeautifulSoup 
import requests
from urllib.parse import unquote
from urllib.parse import urljoin

In [3]:
e = scrape.get(entry)
soup_main = BeautifulSoup(e.text,'html.parser')

In [4]:
#get the links
main_link = soup_main.find_all('a',class_='nav-a')

In [5]:
links=[]
for main in main_link:
    links.append(main.get('href'))

In [6]:
#select those links that start with '/'
collected = []
for link in links[3:]:
    temp = urljoin(entry,link)
    collected.append(temp)
        
print('There are total {} links collected from amazon main page'.format(len(collected)))

There are total 32 links collected from amazon main page


In [7]:
#There is a pattern to the links. End of every link is having the clue to where the link is heading.
main_dict = {}
for links in collected:
    temp = links.split('=')
    #split the link, take the heading
    main_dict[temp[-1]] = links

In [8]:
main_dict.keys()

dict_keys(['nav_orders_first', 'nav_cart', 'nav_cs_bestsellers', 'nav_cs_mobiles', 'nav_cs_gb', 'nav_cs_help', 'nav_cs_electronics', 'nav_cs_primelink_nonmember', 'nav_cs_books', 'nav_cs_fashion', 'nav_cs_newreleases', 'nav_cs_home', 'nav_cs_apay', 'nav_cs_pc', 'nav_cs_coupons', 'nav_cs_toys', 'nav_cs_sell', 'nav_cs_gc', 'nav_cs_automotive', 'nav_cs_beauty', 'nav_cs_grocery', 'nav_cs_video_games', 'nav_cs_sports', 'nav_cs_hpc', 'nav_cs_baby', 'nav_cs_pets', 'nav_cs_kindle_books', 'nav_cs_hi', 'nav_cs_giftfinder', 'nav_cs_amazonbasics', 'nav_cs_audible', 'nav_cs_sns'])

In [9]:
#There are couple of links that lead to personal areas, that require sign-in. Poping them from the dict
main_dict.pop('nav_orders_first')
main_dict.pop('nav_cart')
main_dict.pop('nav_cs_primelink_nonmember')
main_dict.pop('nav_cs_help')
main_dict.pop('nav_cs_sns')
main_dict.pop('nav_cs_apay')

'https://www.amazon.in/gp/sva/dashboard?ref_=nav_cs_apay'

In [10]:
#making dataframe from the dictionary of links
link_collector = pd.DataFrame(list(main_dict.items()),columns=['topics','links'])

In [11]:
#Creating headers so the Amazon website can be accessed
my_headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 ' + ' (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36',
'referer':'http://localhost:8888/'    
}

In [12]:
link_collector.links[0]

'https://www.amazon.in/gp/bestsellers/?ref_=nav_cs_bestsellers'

In [13]:
#The below routine fails since the headers don't work with Amazon correctly. So not very useful, 
link_collector_dict = {} #dict to hold the list of links collected from each links


for index, crawl in enumerate(link_collector.links):
    link_hrf = []
    #print(index,crawl)
    lev = scrape.get(crawl,headers=my_headers)
    lev_soup = BeautifulSoup(lev.text,'html.parser')
    for hrf in lev_soup.find_all('a', attrs={'class':'a-link-normal'}):     
        temp = hrf.get('href')
        link_hrf.append(temp)

    link_collector_dict[link_collector.topics[index]] = link_hrf

In [14]:
link_list_df = pd.DataFrame(list(link_collector_dict.items()),columns=['topics','links_list'])
link_list_df['link_len'] = link_list_df.links_list.apply(lambda x: len(x))
link_list_df = link_list_df[link_list_df.link_len > 60]

In [15]:
link_list_df

,topics,links_list,link_len
0,nav_cs_bestsellers,"[/gp/bestsellers/luggage, /Priority-Disney-Pri...",146
6,nav_cs_newreleases,"[/gp/new-releases/luggage, /WILDHORN-Leather-S...",144


Out of 28 links only 3 pages loaded with default referer and settings. Rest of page needs more info. 
I am planning to go ahead scrape the links under bestsellers first

1) Start by making the list of links under best sellers as full links and see if they provide the content

2) If it provides the content, then check the tags, class, id that can be used to get the data.

3) Create the dataframe and store the data directly in the dataframe.

learn more about the usage

driver.execute_script("window.scrollBy(0, document.body.scrollHeight-400)")

In [40]:
#Initiating the Selenium Drivers
from selenium import webdriver
import time as time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.options import Options
import warnings
warnings.filterwarnings('ignore')

In [88]:
driver = webdriver.Chrome()
driver.get('https://www.linkedin.com')
driver.maximize_window()
driver.find_element_by_class_name('nav__button-secondary').click()
driver.find_element_by_id('username').send_keys('kamaljp@gmail.com')
driver.find_element_by_id('password').send_keys('SepdL!n7')
driver.find_element_by_class_name('login__form_action_container').click()
driver.find_element_by_id('ember19').click()

In [89]:
conn_tot = driver.find_element_by_class_name('pl3').text

In [90]:
driver.find_element_by_class_name('pl3').click()

In [91]:
conn_tot = int(conn_tot)
tot_scroll = int(np.floor(conn_tot/5))
tot_scroll

191

In [92]:
for i in range(1,10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    print(i)
    if i==4:
        driver.execute_script("window.scrollTo(document.body.scrollHeight,window.innerHeight);")
    else:
        continue
html_source = driver.page_source
soup_net = BeautifulSoup(html_source,'html.parser')

1
2
3
4
5
6
7
8
9


In [93]:
t1 = soup_net.find_all('li',attrs={'class':'mn-connection-card artdeco-list'})

In [113]:
t1[0].select('a')[1].get('href')

'/in/dhivya-perumal-a0198b1b/'

In [114]:
t1[0].select('span')[2].get_text().strip()

'Dhivya Perumal'

In [115]:
t1[0].select('span')[4].get_text().strip()

'Assistant Consultant at Tata Consultancy Services'

In [116]:
def get_connLevel_data(beauty_soup, df):
    trial_L1 = beauty_soup.find_all('li',attrs={'class':'mn-connection-card artdeco-list'})
    start = len(df)
    for index,trial in enumerate(trial_L1):
        df.loc[start+index,'Name'] = trial.select('span')[2].get_text().strip()
        df.loc[start+index,'Occupation'] = trial.select('span')[4].get_text().strip()
        df.loc[start+index,'Link'] = trial.select('a')[1].get('href')
    end = len(df)
    print('Total links got is {}'.format(end-start))

In [117]:
data_conn_linkedin = pd.DataFrame()
get_connLevel_data(soup_net,data_conn_linkedin)
data_conn_linkedin.head()

Total links got is 279


,Name,Occupation,Link
0,Dhivya Perumal,Assistant Consultant at Tata Consultancy Services,/in/dhivya-perumal-a0198b1b/
1,Lakshmana Kumar,Assistant Professor at Sona College of Technology,/in/lakshmana-kumar-9b80aab/
2,karthikeyan Balasubramani,Deputy Delivery Manager [SDS2/Tekla] at TRC Wo...,/in/karthikeyan-balasubramani-a3180458/
3,Hitansha Bhatia (she/her),Pastry Chef | French Tutor | Psychology Enthus...,/in/hitansha-bhatia-she-her-b69252187/
4,Prakhar Chauhan (PC),Building state-of-the-art in patients safety a...,/in/prakhar-chauhan-pc-3aa10417/


In [17]:
#Creating a subroutine to get page
def get_page(main_driver):
    element = main_driver.find_element_by_class_name('a-last')
    main_driver.execute_script("arguments[0].scrollIntoView();",element)
    time.sleep(5)
    soup = main_driver.page_source
    trial = BeautifulSoup(soup,'html.parser')
    return trial

In [18]:
#get the pages that are still un-selected. These many clicks will be required to traverse the site
def get_page_avbl(page_soup):
    return len(page_soup.find_all('li',attrs={'class':'a-normal'}))

In [20]:
def crawler(page_url):
    driver = webdriver.Chrome()
    driver.get(page_url)
    driver.maximize_window()
    stor_dataframe = pd.DataFrame()
    #Above code initialises the selenium driver and opens the browser
    
    firstpage = get_page(driver) #gets the first page
    get_pagelevel_data(firstpage,stor_dataframe) # gets the data in the first page
    page_avbl = get_page_avbl(firstpage) #gets the number of pages
    print('This url contains {} pages'.format(page_avbl+1))
    for page in range(page_avbl):#looping the pages available
        print('Moving to {} page for scraping'.format(page+2))
        try:
            driver.find_element_by_class_name('a-last').click() #moving to the next page
            page_next = get_page(driver) #getting next page data
            get_pagelevel_data(page_next,stor_dataframe) # collecting the next page data
        except NoSuchElementException as nse:
            print('Element not found, breaking')
    print('ending crawler for {} and returning payload'.format(page_url))
    print('Links collected is {} and quitting driver'.format(stor_dataframe.shape[0]))
    driver.quit()
    return stor_dataframe #returning the dataframe

In [40]:
#Even though the crawler is headless, the printing messages will feedback on the status
def headless_crawler(page_url):
    options = Options()
    options.headless = True #making this driver headless so it can work in the background
    driver = webdriver.Chrome(options=options, executable_path=r'chromedriver.exe')
    driver.get(page_url)
    driver.maximize_window()
    stor_dataframe = pd.DataFrame()
    #Above code initialises the selenium driver and opens the browser
    
    firstpage = get_page(driver) #gets the first page
    get_pagelevel_data(firstpage,stor_dataframe) # gets the data in the first page
    page_avbl = get_page_avbl(firstpage) #gets the number of pages
    print('This url contains {} pages'.format(page_avbl+1))
    for page in range(page_avbl):#looping the pages available
        print('Moving to {} page for scraping'.format(page+2))
        try:
            driver.find_element_by_class_name('a-last').click() #moving to the next page
            page_next = get_page(driver) #getting next page data
            get_pagelevel_data(page_next,stor_dataframe) # collecting the next page data
        except NoSuchElementException as nse:
            print('Element not found, breaking')
    print('ending crawler for {} and returning payload'.format(page_url))
    print('Links collected is {} and quitting driver'.format(stor_dataframe.shape[0]))
    driver.quit()
    return stor_dataframe #returning the dataframe

In [41]:
vid_url = 'https://www.amazon.in/gp/bestsellers/videogames'

In [42]:
#The headless works.... :D
vid_games = headless_crawler(vid_url)

C:\WorkFiles\WPy64-37120\pypy3.7-v7.3.7-win64\site-packages\ipykernel_launcher.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """
C:\WorkFiles\WPy64-37120\pypy3.7-v7.3.7-win64\site-packages\ipykernel_launcher.py:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  This is separate from the ipykernel package so we can avoid doing imports until


Total data got is 50
This url contains 2 pages
Moving to 2 page for scraping


C:\WorkFiles\WPy64-37120\pypy3.7-v7.3.7-win64\site-packages\ipykernel_launcher.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


Total data got is 50
ending crawler for https://www.amazon.in/gp/bestsellers/videogames and returning payload
Links collected is 100 and quitting driver


In [43]:
vid_games.head()

,Name,reviews,stars,price,link_in
0,"AmazonBasics Extended Gaming Mouse Pad,Black","35,808",4.5 out of 5 stars,₹469.00,/AmazonBasics-Extended-Gaming-Mouse-Black/dp/B...
1,Rs.1000 Sony PlayStation Network Wallet Top-Up...,4.7 out of 5 stars,Sony Interactive Entertainment Europe,"3,115",/Rs-1000-Sony-PlayStation-Network-Delivery/dp/...
2,Rs.500 Sony PlayStation Network Wallet Top-Up ...,4.7 out of 5 stars,Sony Interactive Entertainment Europe,"3,107",/Sony-PlayStation-Network-Delivery-Digital/dp/...
3,"JBL Quantum 100, Wired Over Ear Gaming Headpho...","3,473",4.0 out of 5 stars,"₹2,299.00",/JBL-Quantum-Over-Ear-Headset-Detachable/dp/B0...
4,Cosmic Byte GS410 Wired Over-ear Headphones wi...,4.0 out of 5 stars,Cosmic Byte,"18,977",/Cosmic-Byte-Headphones-Laptop-Android/dp/B074...


Next step is to join the URLs and get the product level details. 

Join the link of each product, by creating the function for that purpose and send the created dataframe

scrape the product page of the information

store it back in the initial dataframe itself

In [23]:
#creating full_linkins for products
def f_link_create(df):
    for index,links in enumerate(df.link_in):
        #print(links)
        df.loc[index,'f_link'] = urljoin('https://www.amazon.in/',links)

    return df

In [12]:
vid_games = f_link_create(vid_games)

AttributeError: 'DataFrame' object has no attribute 'link_in'

In [22]:
def crawler_product(page_url,df,indexF):
    driver = webdriver.Chrome()
    driver.get(page_url)
    driver.maximize_window()
    prod_soup = driver.page_source
    prod_nugs = BeautifulSoup(prod_soup,'html.parser')
   
    #Populating the dataframe directly with the below commands. The details were found using 
    table_prod_1 = prod_nugs.find('table',attrs={'id':'productDetails_techSpec_section_1'})
    table_prod_2 = prod_nugs.find('table',attrs={'id':'productDetails_detailBullets_sections1'})
    review_table = prod_nugs.find_all('table',attrs={'id':'histogramTable'})[1]
    if table_prod_1 == None or table_prod_2 == None :
        review_table = prod_nugs.find_all('table',attrs={'id':'histogramTable'})[1]
            #Scraping of the data from the tables
        df.loc[indexF,'brand'] = 'voucher_or_program'
        df.loc[indexF,'color'] = 'voucher_or_program'
        df.loc[indexF,'Pdt_dimension'] = 'voucher_or_program'
        df.loc[indexF,'Weight'] = 'voucher_or_program'
        df.loc[indexF,'origin'] = 'voucher_or_program'
        df.loc[indexF,'ASIN'] = 'voucher_or_program'
        df.loc[indexF,'category'] = 'voucher_or_program'
        df.loc[indexF,'5Star'] = review_table.find_all('tr')[0].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
        df.loc[indexF,'4Star'] = review_table.find_all('tr')[1].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
        df.loc[indexF,'3Star'] = review_table.find_all('tr')[2].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
        df.loc[indexF,'2Star'] = review_table.find_all('tr')[3].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
        df.loc[indexF,'1Star'] = review_table.find_all('tr')[4].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
    else:
        #Scraping of the data from the tables
        df.loc[indexF,'brand'] = table_prod_1.find_all('td')[0].get_text().strip()
        df.loc[indexF,'color'] = table_prod_1.find_all('td')[2].get_text().strip()
        df.loc[indexF,'Pdt_dimension'] = table_prod_2.find_all('td')[-2].get_text().strip()
        df.loc[indexF,'Weight'] = table_prod_2.find_all('td')[-1].get_text().strip()
        df.loc[indexF,'origin'] = table_prod_1.find_all('td')[-1].get_text().strip()
        df.loc[indexF,'ASIN'] = table_prod_2.find_all('td')[0].get_text().strip()
        df.loc[indexF,'category'] = table_prod_2.find_all('td')[-1].get_text().strip()
        df.loc[indexF,'5Star'] = review_table.find_all('tr')[0].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
        df.loc[indexF,'4Star'] = review_table.find_all('tr')[1].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
        df.loc[indexF,'3Star'] = review_table.find_all('tr')[2].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
        df.loc[indexF,'2Star'] = review_table.find_all('tr')[3].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
        df.loc[indexF,'1Star'] = review_table.find_all('tr')[4].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()
    
    print('Completed the data collection')
    driver.quit()

In [200]:
for indexF,prod_link in enumerate(vid_games.f_link[94:]):
    crawler_product(prod_link,vid_games,indexF)

Completed the data collection
Completed the data collection
Completed the data collection
Completed the data collection
Completed the data collection
Completed the data collection


In [187]:
review_table.find_all('tr')[0].find_all('span',attrs={'class':'a-size-base'})[1].get_text().strip()

'66%'

In [201]:
vid_games[vid_games.brand.isna()]

,Name,reviews,stars,price,link_in,f_link,brand,color,Pdt_dimension,Weight,origin,ASIN,category,5Star,4Star,3Star,2Star,1Star
94,Sony Horizon: Zero Dawn - Complete Edition (Pl...,4.5 out of 5 stars,Sony,"1,528",/Horizon-Zero-Dawn-Complete-PS4/dp/B07D2NY7RW/...,https://www.amazon.in/Horizon-Zero-Dawn-Comple...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,Cosmic Byte GS410 Headphones with Mic and for ...,4.1 out of 5 stars,Cosmic Byte,"18,705",/Cosmic-Byte-GS410-Headphones-Grey/dp/B07K7XRJ...,https://www.amazon.in/Cosmic-Byte-GS410-Headph...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Microsoft Xbox Series X/S Wireless Controller ...,4.3 out of 5 stars,Microsoft,255,/Microsoft-Wireless-Controller-USB-C-Cable/dp/...,https://www.amazon.in/Microsoft-Wireless-Contr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Nitho Switch Armour Case Lite for Nintendo Swi...,4.5 out of 5 stars,Nitho,19,/Nitho-NITHO-NSW-ACLT-K-Switch-Armour-Case/dp/...,https://www.amazon.in/Nitho-NITHO-NSW-ACLT-K-S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,"Blue Yeti USB Microphone for Recording, Stream...","15,241",4.3 out of 5 stars,"₹10,995.00",/Blue-Yeti-USB-Microphone-Midnight/dp/B01LY6Z2...,https://www.amazon.in/Blue-Yeti-USB-Microphone...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,Marvel's Spider Man (PS4),4.6 out of 5 stars,Sony,"2,837",/Sony-195229-Marvels-Spider-Man/dp/B07CR4GQD3/...,https://www.amazon.in/Sony-195229-Marvels-Spid...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [194]:
vid_games[vid_games.f_link == 'https://www.amazon.in/Call-Duty-Infinite-Warfare-PS4/dp/B01JG2VZFQ/ref=zg_bs_videogames_58/260-6571345-4071009?pd_rd_i=B01JG2VZFQ&psc=1']

,Name,reviews,stars,price,link_in,f_link,brand,color,Pdt_dimension,Weight,origin,ASIN,category,5Star,4Star,3Star,2Star,1Star
57,Call of Duty: Infinite Warfare (PS4),4.2 out of 5 stars,ACTIVISION,"1,876",/Call-Duty-Infinite-Warfare-PS4/dp/B01JG2VZFQ/...,https://www.amazon.in/Call-Duty-Infinite-Warfa...,diff_product,diff_product,diff_product,diff_product,diff_product,diff_product,diff_product,57%,21%,10%,5%,7%


In [45]:
driver.quit()